# Set up Environment

In [5]:
import gymnasium as gym
import random

In [ ]:
%pip install atari

In [6]:
env = gym.make('CartPole-v1', render_mode = "human")
states = env.observation_space.shape # didn't isolate 4
actions = env.action_space.n

In [3]:
# We need to find shape fo states

In [7]:
states

(4,)

In [26]:
states

(4,)

In [5]:
actions # Two directions of movement 

2

## Loading different Envirnoment and finding actions

In [4]:
env = gym.make("ALE/Adventure-v5")
states = env.observation_space.shape[0]
actions = env.ction_space.n

NamespaceNotFound: Namespace ALE not found. Have you installed the proper package for ALE?

## Continuing on

In [30]:
episodes = 10

for episode in range(1, episodes+1):
    obeservation, info = env.reset()
    done = False
    score = 0
    
    while not done:
        env.render()
        action = random.choice([0,1])
        observation, reward, terminated, truncated, info= env.step(action)
        env.close()
        
    print(episode, score)
    
    

error: display Surface quit

In [1]:
import gym
env = gym.make("CartPole-v0", render_mode="human")
env.action_space.seed(42)
score = 0

observation, info = env.reset(seed=42)

episodes = 20

for _ in range(episodes):
    observation, reward, terminated, truncated, info = env.step(env.action_space.sample())
    

    if terminated or truncated:
        observation, info = env.reset()
        
    score =+ reward

    print(_, score)

env.close()

/Users/rish/opt/anaconda3/lib/python3.9/site-packages/gym/envs/registration.py:555: UserWarning: WARN: The environment CartPole-v0 is out of date. You should consider upgrading to version `v1`.
  logger.warn(


0 1.0
1 1.0
2 1.0
3 1.0
4 1.0
5 1.0
6 1.0
7 1.0
8 1.0
9 1.0
10 1.0
11 1.0
12 1.0
13 1.0
14 1.0
15 1.0
16 1.0
17 1.0
18 1.0
19 1.0


In [6]:



episodes = 10
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0 
    
    while not done:
        env.render()
        action = random.choice([0,1])
        n_state, reward, done, _, info = env.step(action)
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))
    


Episode:1 Score:45.0
Episode:2 Score:21.0
Episode:3 Score:11.0
Episode:4 Score:22.0
Episode:5 Score:28.0
Episode:6 Score:14.0
Episode:7 Score:30.0
Episode:8 Score:24.0
Episode:9 Score:30.0
Episode:10 Score:12.0


In [9]:
state

(array([-0.02587322,  0.00634657,  0.00610725,  0.02581345], dtype=float32),
 {})

# Model

In [2]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
import numpy as np
from tensorflow.keras.optimizers import Adam

In [1]:
def build_model(states, actions):
    model = Sequential()
    model.add(Flatten(input_shape=(1,) + states)) # Edited states
    model.add(Dense(24, activation='relu'))
    model.add(Dense(24, activation='relu'))
    
    model.add(Dense(actions, activation='linear'))
    return model

In [8]:
model = build_model(states, actions)

2023-03-12 16:14:46.884433: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [9]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 4)                 0         
                                                                 
 dense (Dense)               (None, 24)                120       
                                                                 
 dense_1 (Dense)             (None, 24)                600       
                                                                 
 dense_2 (Dense)             (None, 2)                 50        
                                                                 
Total params: 770
Trainable params: 770
Non-trainable params: 0
_________________________________________________________________


# Making Agent 

In [10]:
from rl.agents import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory

In [11]:
model = build_model(states, actions)

In [12]:
def build_agent(model, actions):
    policy = BoltzmannQPolicy()
    memory = SequentialMemory(limit=50000, window_length=1)
    dqn = DQNAgent(model=model, memory=memory, policy=policy, 
                  nb_actions=actions, nb_steps_warmup=10, target_model_update=1e-2)
    return dqn

Something with the end shape of the model is weird

In [13]:
# Compiling
dqn = build_agent(model, actions)
dqn.compile(Adam(learning_rate = 1e-3), metrics = ["mae"])
dqn.fit(env, nb_steps = 50000, visualize = False, verbose = 1)

2023-03-12 16:15:36.766181: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:354] MLIR V1 optimization pass is not enabled


Training for 50000 steps ...
Interval 1 (0 steps performed)


ValueError: Error when checking input: expected flatten_1_input to have shape (1, 4) but got array with shape (1, 2)

: 

In [ ]:
# Will fix shape error 

There is someting wrong with the dimension of states, it was expected to be (1,2) but instead its (1,4)

We can also just not isolate the states[0] at the begining and then just do it in the flatten layer

https://github.com/nicknochnack/TensorflowKeras-ReinforcementLearning/issues/1